In [ ]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from AnimalShelter import AnimalShelter

username = "aacuser"
password = "Waverly1"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read_all({}))

app = JupyterDash('SimpleExample')

image_filename = 'graz64.png' # ive resized this image to be 64x64
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),html.Center(html.B(html.H2('Branden McCusker'))),
    html.Hr(),
    html.Div(

    className='row',
    style={'display': 'flex'},
    children=[
        html.Button(id='buttonWater',n_clicks=0, children= 'Water Rescue'),
        html.Button(id='buttonMountain',n_clicks=0, children= 'Mountain Rescue'),
        html.Button(id='buttonDisaster',n_clicks=0, children='Disaster Rescue'),
        html.Button(id='buttonReset', n_clicks=0, children='Reset') 
        ]
    ),

    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],

        data=df.to_dict('records'),

        page_size=100,
        style_table={'height':'400px','overflowY':'auto','overflowX':'auto'},
        style_header={
            'backgroundColor':'rgb(240,230,230)',
            'fontWeight':'bold' 
        },

        style_data={
            'whiteSpace':'normal',
            'height':'auto'
        },

        tooltip ={i: {
            'value': i,
            'use_with': 'both' 
        }
        for i in df.columns},
        tooltip_delay=0,
        tooltip_duration = None,
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable=True,
        row_selectable='single',
        row_deletable=False,
        selected_rows=[],
    ),
    html.Br(),
    html.Hr(),

    html.Div(className='row',
    style={'display' : 'flex'},
    children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ])
])

@app.callback([Output('datatable-id','data')],
[Input('buttonWater', 'n_clicks'),Input('buttonMountain','n_clicks'), 
Input('buttonDisaster','n_clicks'),Input('buttonReset','n_clicks')])
def update_dashboard(buttonWater,buttonMountain,buttonDisaster,buttonReset):
    if (int(buttonWater) >= 1):
        df = pd.Dataframe.from_records(shelter.read_all({'$and': [ 
        {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
        {'breed':'Newfoundland'}]}, 
        {'sex_upon_outcome':'Intact Female'}, {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
        buttonMountain, buttonDisaster, buttonReset = 0   
    elif (int(buttonMountain)>= 1):
        df = pd.Dataframe.from_records(shelter.read_all({'$and': [ 
        {'$or': [ {'breed':'German Shepherd'}, {'breed':'Alaskan Malamute'},
        {'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]}, 
        {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
        buttonWater, buttonDisaster ,buttonReset = 0
    elif (int(buttonDisaster)>=1):
        df = pd.Dataframe.from_records(shelter.read_all({'$and': [ 
        {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
        {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]}, 
        {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':20, 'gte':300}}]}))
        bt1, buttonMountain, buttonReset = 0
    elif(int(buttonReset)>=1):
        df = pd.Dataframe.from_records(shelter.read_all())
        bt1, buttonMountain, buttonDisaster = 0
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        
    data=df.to_dict('records') 
    return data


@app.callback(
Output('datatable-id', 'style_data_conditional'),
[Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
Output('graph-id', "children"),
[Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph( 
            figure = px.pie(df, values=values, names=names, title='Percentage of breeds available')
        ) 
    ]


@app.callback(
Output('map-id', "children"),
[Input('datatable-id', "derived_viewport_data"),
Input('datatable-id',"derived_viewport_selected_rows")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], zoom=15, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
            dl.Tooltip(dff['breed']), 
            dl.Popup([
            html.H1("Animal Name"),
            html.P(dff.loc[0,'name'])
            ])
        ])
    ])
]

app